In [50]:
#WALMART STORE PREDICTION - FORECASTING

In [51]:
#Importing the required libraries
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import calendar
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
import datetime

In [52]:
#Importing provided train data as data
data=pd.read_csv("trainMerged.csv")

In [53]:
data.head()

,Store,Date,IsHoliday,Dept,Weekly_Sales,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
0,10,2010-02-05,False,1,40212.84,B,126512,54.34,2.962,NaN,NaN,NaN,NaN,NaN,126.442065,9.765
1,10,2010-02-05,False,98,74.00,B,126512,54.34,2.962,NaN,NaN,NaN,NaN,NaN,126.442065,9.765
2,10,2010-02-05,False,32,26116.73,B,126512,54.34,2.962,NaN,NaN,NaN,NaN,NaN,126.442065,9.765
3,10,2010-02-05,False,25,20779.50,B,126512,54.34,2.962,NaN,NaN,NaN,NaN,NaN,126.442065,9.765
4,10,2010-02-05,False,34,50585.42,B,126512,54.34,2.962,NaN,NaN,NaN,NaN,NaN,126.442065,9.765


In [54]:
data.describe()

,Store,Dept,Weekly_Sales,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
count,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,150681.000000,111248.000000,137091.000000,134967.000000,151432.000000,421570.000000,421570.000000
mean,22.200546,44.260317,15981.258123,136727.915739,60.090059,3.361027,7246.420196,3334.628621,1439.421384,3383.168256,4628.975079,171.201947,7.960289
std,12.785297,30.492054,22711.183519,60980.583328,18.447931,0.458515,8291.221345,9475.357325,9623.078290,6292.384031,5962.887455,39.159276,1.863296
min,1.000000,1.000000,-4988.940000,34875.000000,-2.060000,2.472000,0.270000,-265.760000,-29.100000,0.220000,135.160000,126.064000,3.879000
25%,11.000000,18.000000,2079.650000,93638.000000,46.680000,2.933000,2240.270000,41.600000,5.080000,504.220000,1878.440000,132.022667,6.891000
50%,22.000000,37.000000,7612.030000,140167.000000,62.090000,3.452000,5347.450000,192.000000,24.600000,1481.310000,3359.450000,182.318780,7.866000
75%,33.000000,74.000000,20205.852500,202505.000000,74.280000,3.738000,9210.900000,1926.940000,103.990000,3595.040000,5563.800000,212.416993,8.572000
max,45.000000,99.000000,693099.360000,219622.000000,100.140000,4.468000,88646.760000,104519.540000,141630.610000,67474.850000,108519.280000,227.232807,14.313000


In [55]:
data['Date']=pd.to_datetime(data['Date'])

In [56]:
data.isnull().sum(axis=0).sort_values(ascending=False)

MarkDown2       310322
MarkDown4       286603
MarkDown3       284479
MarkDown1       270889
MarkDown5       270138
Unemployment         0
CPI                  0
Fuel_Price           0
Temperature          0
Size                 0
Type                 0
Weekly_Sales         0
Dept                 0
IsHoliday            0
Date                 0
Store                0
dtype: int64

In [57]:
#Replacing Null values with 0
data=data.fillna(0)

In [58]:
data['Month'] = pd.DatetimeIndex(data['Date']).month
data['Month'] = data['Month'].apply(lambda x: calendar.month_abbr[x])
data['Year'] = pd.DatetimeIndex(data['Date']).year
data['Day']=pd.DatetimeIndex(data['Date']).day

In [59]:
#Creating Dummy Variables for Month Column
data=data.join(pd.get_dummies(data.Month))

In [60]:
data.update(data.drop("Month",axis=1,inplace=True))

In [61]:
data['IsHoliday']=data['IsHoliday'].map({"True": 1, "False": 0})

In [62]:
data=data.join(pd.get_dummies(data.Type,prefix="Type_"))

In [63]:
data.update(data.drop("Type",axis=1,inplace=True))

In [64]:
data['Super_Bowl']=np.where(((data['Date']==datetime.datetime(2010, 2, 12)) | (data['Date']==datetime.datetime(2011, 2, 11)) | (data['Date']==datetime.datetime(2012, 2, 10)) | (data['Date']==datetime.datetime(2013, 2, 8))),1,0)
data['Labour_Day']=np.where((data['Date']==datetime.datetime(2010, 9, 10)) | (data['Date']==datetime.datetime(2011, 9, 9)) | (data['Date']==datetime.datetime(2012, 9, 7)) | (data['Date']==datetime.datetime(2013, 9, 6)),1,0)
data['Thanksgiving']=np.where((data['Date']==datetime.datetime(2010, 11, 26)) | (data['Date']==datetime.datetime(2011, 11, 25)) | (data['Date']==datetime.datetime(2012, 11, 23)) | (data['Date']==datetime.datetime(2013, 11, 29)),1,0)
data['Christmas']=np.where((data['Date']==datetime.datetime(2010, 12, 31)) | (data['Date']==datetime.datetime(2011, 12, 30)) | (data['Date']==datetime.datetime(2012, 12, 28)) | (data['Date']==datetime.datetime(2013, 12, 27)),1,0)

In [65]:
data['IsHoliday']=data['IsHoliday']|data['Super_Bowl']|data['Labour_Day']|data['Thanksgiving']|data['Christmas']

In [66]:
data=data.drop(columns=["Super_Bowl","Labour_Day","Thanksgiving","Christmas","Date"])

In [67]:
data.head()

,Store,IsHoliday,Dept,Weekly_Sales,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,...,Jul,Jun,Mar,May,Nov,Oct,Sep,Type__A,Type__B,Type__C
0,10,False,1,40212.84,126512,54.34,2.962,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
1,10,False,98,74.00,126512,54.34,2.962,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
2,10,False,32,26116.73,126512,54.34,2.962,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
3,10,False,25,20779.50,126512,54.34,2.962,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
4,10,False,34,50585.42,126512,54.34,2.962,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0


In [68]:
#Defining weekly sales as our target variable "y"
y=data["Weekly_Sales"]

In [69]:
data.drop("Weekly_Sales",inplace=True, axis=1)
#Since Weekly Sales is our target variable, it shall cease to exist in data

In [70]:
#Modeling
x_train, x_test, y_train, y_test= train_test_split(data,y,test_size=0.2,random_state=0)

In [71]:
model = RandomForestRegressor(n_estimators=100, n_jobs=-1)
model.fit(x_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [72]:
y_pred=model.predict(x_test)

In [73]:
y_pred=pd.DataFrame(data=y_pred,index=x_test.index)

In [74]:
model.score(x_test,y_test)*100

97.72246005974526

In [75]:
#Loading data for which predictions are to be made

In [76]:
testdata=pd.read_csv("testMerged.csv")
testdata.head()

,Store,Date,IsHoliday,Dept,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
0,10,2012-11-02,False,1,B,126512,70.79,4.099,25680.2,6037.06,44.68,17412.04,4223.05,131.236226,6.943
1,10,2012-11-02,False,72,B,126512,70.79,4.099,25680.2,6037.06,44.68,17412.04,4223.05,131.236226,6.943
2,10,2012-11-02,False,71,B,126512,70.79,4.099,25680.2,6037.06,44.68,17412.04,4223.05,131.236226,6.943
3,10,2012-11-02,False,21,B,126512,70.79,4.099,25680.2,6037.06,44.68,17412.04,4223.05,131.236226,6.943
4,10,2012-11-02,False,67,B,126512,70.79,4.099,25680.2,6037.06,44.68,17412.04,4223.05,131.236226,6.943


In [77]:
testdata.describe()

,Store,Dept,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
count,115064.000000,115064.000000,115064.000000,115064.000000,115064.000000,114915.000000,86437.000000,105235.000000,102176.000000,115064.000000,76902.000000,76902.000000
mean,22.238207,44.339524,136497.688921,53.941804,3.581546,7689.216439,3734.051729,2403.088666,3356.219071,3922.681189,176.961347,6.868733
std,12.809930,30.656410,61106.926438,18.724153,0.239442,10698.760716,8323.495014,13767.939313,7570.501545,19445.150745,41.239967,1.583427
min,1.000000,1.000000,34875.000000,-7.290000,2.872000,-2781.450000,-35.740000,-179.260000,0.220000,-185.170000,131.236226,3.684000
25%,11.000000,18.000000,93638.000000,39.820000,3.431000,1966.460000,180.350000,15.100000,155.460000,1309.300000,138.402033,5.771000
50%,22.000000,37.000000,140167.000000,54.470000,3.606000,4842.290000,742.590000,78.260000,840.940000,2390.430000,192.304445,6.806000
75%,33.000000,74.000000,202505.000000,67.350000,3.766000,9439.140000,2735.670000,272.580000,3096.920000,4227.270000,223.244532,8.036000
max,45.000000,99.000000,219622.000000,101.950000,4.125000,103184.980000,71074.170000,149483.310000,65344.640000,771448.100000,228.976456,10.199000


In [78]:
test=testdata

In [79]:
testdata.isnull().sum(axis=0).sort_values(ascending=False)

Unemployment    38162
CPI             38162
MarkDown2       28627
MarkDown4       12888
MarkDown3        9829
MarkDown1         149
MarkDown5           0
Fuel_Price          0
Temperature         0
Size                0
Type                0
Dept                0
IsHoliday           0
Date                0
Store               0
dtype: int64

In [80]:
#Replacing empty CPI and Unemployment values with mean
testdata['CPI']=testdata.groupby(['Dept'])['CPI'].transform(lambda x: x.fillna(x.mean()))
testdata['Unemployment']=testdata.groupby(['Dept'])['Unemployment'].transform(lambda x: x.fillna(x.mean()))

In [81]:
#Replacing null values with 0
testdata=testdata.fillna(0)

In [82]:
testdata['Date']=pd.to_datetime(testdata['Date'])

In [83]:
testdata['Month']=pd.DatetimeIndex(testdata['Date']).month
testdata['Month']=testdata['Month'].apply(lambda x: calendar.month_abbr[x])
testdata['Year']=pd.DatetimeIndex(testdata['Date']).year
testdata['Day']=pd.DatetimeIndex(testdata['Date']).day

In [84]:
testdata=(testdata.join(pd.get_dummies(testdata.Month)))

In [85]:
testdata.update(testdata.drop("Month",axis=1,inplace=True))

In [86]:
testdata['IsHoliday']=testdata['IsHoliday'].map({"True": 1, "False": 0})

In [87]:
testdata=testdata.join(pd.get_dummies(testdata.Type,prefix="Type_"))

In [88]:
testdata.update(testdata.drop("Type",axis=1,inplace=True))

In [89]:
testdata['Super_Bowl']=np.where(((testdata['Date']==datetime.datetime(2010, 2, 12)) | (testdata['Date']==datetime.datetime(2011, 2, 11)) | (testdata['Date']==datetime.datetime(2012, 2, 10)) | (testdata['Date']==datetime.datetime(2013, 2, 8))),1,0)
testdata['Labour_Day']=np.where((testdata['Date']==datetime.datetime(2010, 9, 10)) | (testdata['Date']==datetime.datetime(2011, 9, 9)) | (testdata['Date']==datetime.datetime(2012, 9, 7)) | (testdata['Date']==datetime.datetime(2013, 9, 6)),1,0)
testdata['Thanksgiving']=np.where((testdata['Date']==datetime.datetime(2010, 11, 26)) | (testdata['Date']==datetime.datetime(2011, 11, 25)) | (testdata['Date']==datetime.datetime(2012, 11, 23)) | (testdata['Date']==datetime.datetime(2013, 11, 29)),1,0)
testdata['Christmas']=np.where((testdata['Date']==datetime.datetime(2010, 12, 31)) | (testdata['Date']==datetime.datetime(2011, 12, 30)) | (testdata['Date']==datetime.datetime(2012, 12, 28)) | (testdata['Date']==datetime.datetime(2013, 12, 27)),1,0)

In [90]:
testdata['IsHoliday']=testdata['IsHoliday']|testdata['Super_Bowl']|testdata['Labour_Day']|testdata['Thanksgiving']|testdata['Christmas']

In [91]:
testdata=testdata.drop(columns=["Super_Bowl","Labour_Day","Thanksgiving","Christmas","Date"])

In [92]:
testdata.head()

,Store,IsHoliday,Dept,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,...,Feb,Jan,Jul,Jun,Mar,May,Nov,Type__A,Type__B,Type__C
0,10,False,1,126512,70.79,4.099,25680.2,6037.06,44.68,17412.04,...,0,0,0,0,0,0,1,0,1,0
1,10,False,72,126512,70.79,4.099,25680.2,6037.06,44.68,17412.04,...,0,0,0,0,0,0,1,0,1,0
2,10,False,71,126512,70.79,4.099,25680.2,6037.06,44.68,17412.04,...,0,0,0,0,0,0,1,0,1,0
3,10,False,21,126512,70.79,4.099,25680.2,6037.06,44.68,17412.04,...,0,0,0,0,0,0,1,0,1,0
4,10,False,67,126512,70.79,4.099,25680.2,6037.06,44.68,17412.04,...,0,0,0,0,0,0,1,0,1,0


In [93]:
testdata['Aug']=0
testdata['Sep']=0
testdata['Oct']=0

In [94]:
testdata=testdata[x_test.columns]

In [95]:
#Predicting values for testdata with model retrieved
pred=pd.DataFrame(data=model.predict(testdata),index=testdata.index,columns=["Weekly_Sales"])

In [96]:
pred.index=test["Store"].astype(str)+"_"+test["Dept"].astype(str)+"_"+test["Date"].astype(str)

In [97]:
pred.index.set_names(names="Id",inplace=True)

In [98]:
#Exporting the required predicted values as csv file
pred.to_csv("Predictions.csv")